# Get data
```
mkdir -p /home/ubuntu/environment/long/20210104-fs711-longitudinal
cd /home/ubuntu/environment/long/20210104-fs711-longitudinal
aws s3 cp \
  --recursive \
  --exclude "*" \
  --include "*.stats" \
  s3://cmet-scratch/20210104-fs711-longitudinal .
aws s3 cp \
  s3://cmet-scratch/maclaren-cmeds/demographics.tsv \
  /home/ubuntu/environment/long/20210104-fs711-longitudinal/maclaren
aws s3 cp \
  s3://cmet-thinq/v1.0/datasets/validation/AccuracyNonCortical/demographics.tsv \
  /home/ubuntu/environment/long/20210104-fs711-longitudinal/AccuracyNonCortical
aws s3 cp \
  s3://cmet-thinq/v1.0/datasets/validation/AccuracyNonCortical/demographics.tsv \
  /home/ubuntu/environment/long/20210104-fs711-longitudinal/AccuracyNonCortical-defaced
```

## Split data into seperate subdirs
### To faciliate processing

```
cd /home/ubuntu/environment/long/20210104-fs711-longitudinal/maclaren
mkdir cross
mkdir long
mkdir long-base
mv sub-??_run-?? ./cross/
mv sub-??_base ./long-base
mv sub* ./long/
```

### Then rename the long dirs (gross)

```
cd ./long
for DIR_STEM in `ls -1|sed 's/\..*//'`; do  SUB_NUM=`echo $DIR_STEM|sed 's/sub-//'|sed 's/_run.*//'`; mv ${DIR_STEM}.long.sub-${SUB_NUM}_base ${DIR_STEM}; done
```

In [5]:
import json
import os
import fnmatch
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
# imports find_json_files(); load_json_file(); load_dataset(); load_fs_dataset()
from cmeds import *
# imports calc_cvs(); session_permute(); monte_carlo_perm_test
from test_retest import *

In [19]:
structs_of_interest = [
    'Left-Lateral-Ventricle',
    'Left-Hippocampus',
    'Left-Amygdala',
    'Left-Caudate',
    'Left-Putamen',
    'Right-Lateral-Ventricle',
    'Right-Hippocampus',
    'Right-Amygdala',
    'Right-Caudate',
    'Right-Putamen'
]
# Add some lateral regions together so we can directly compare to table 1 in MacLaren et al.
regions = [             
            [ ['Left-Hippocampus', 'Right-Hippocampus'],'Hippocampus' ],
            [ ['Left-Lateral-Ventricle', 'Right-Lateral-Ventricle'],'Lateral-Ventricles' ],
            [ ['Left-Amygdala', 'Right-Amygdala',],'Amygdala' ],
            [ ['Left-Putamen', 'Right-Putamen'],'Putamen' ],
            [ ['Left-Caudate', 'Right-Caudate'],'Caudate' ],
          ]
drop_subjects = []
demofile = '/home/ubuntu/environment/long/20210104-fs711-longitudinal/maclaren/demographics.tsv'

datadir = '/home/ubuntu/environment/long/20210104-fs711-longitudinal/maclaren/long'
maclaren_fs711_samseglong_df = load_fssamseg_dataset(datadir, demofile, structs_of_interest, drop_subjects=[])
maclaren_fs711_samseglong_df = add_regions(maclaren_fs711_samseglong_df,regions)

Dropping the following subjects []


In [20]:
# The column name that holds session info in the demographics.tsv
session_col='session'
# The column name that holds subject info in the demographics.tsv
subject_col='subject_num'
structs_of_interest = ['Hippocampus', 'Lateral-Ventricles', 'Amygdala', 'Putamen', 'Caudate']


session_list= [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
subject_list= [1,2,3]

n = 1000
perm_test_reults_fs711_samseglong = monte_carlo_perm_test(maclaren_fs711_samseglong_df, subject_list, session_list, subject_col, session_col, structs_of_interest, n_itrs=n, method='gluer')

In [21]:
perm_test_reults_fs711_samseglong

,Hippocampus,Lateral-Ventricles,Amygdala,Putamen,Caudate
mean-vol,8942.957210,15599.331460,3457.148996,11104.434273,7204.101679
total-cov,0.684599,1.637940,1.088511,1.129612,1.017349
session-cov,0.662999,1.204598,0.994030,1.016796,0.923261
abs-diff-cov,0.021600,0.433342,0.094481,0.112816,0.094089
p-vals,0.676000,0.000000,0.188000,0.171000,0.145000
